<a href="https://colab.research.google.com/github/PanoEvJ/Constitutional_BloomZ/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trl

In [6]:
!cp '/content/drive/MyDrive/gpt-j-6b-toxicity.py' train.py

In [ ]:
!python train.py --model_save_path='./'